In [1]:
#!pip install carbontracker
#!pip install albumentations==1.3.0
#!pip install timm
#!pip install segmentation-models-pytorch

In [2]:
from dataset_florian import FashionMNISTDataset, FashionMNISTDatasetRGB
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from models import UNet, UNETR
from tqdm import tqdm
import torch.nn as nn
from PIL import Image
import numpy as np
import torch
import wandb
from carbontracker.tracker import CarbonTracker
import segmentation_models_pytorch as smp
from metrics import LossManager

c:\Users\noegi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# CONSTANTS

DATASET_PATH = 'datasets/'
MODEL_PATH = 'models/'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 10
BATCH_SIZE = 16
INPUT_SIZE = (224, 224, 1)

kwargs = {'encoder_name':'vgg16',        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    'encoder_weights': None,     # use `imagenet` pre-trained weights for encoder initialization
    'in_channels':3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    'classes':10,                      # model output channels (number of classes in your dataset)
}

model = smp.Unet(**kwargs).to(DEVICE)
model.train()
dataset_name = 'data_rgb_b_noise_len_10k'
# LOAD DATASET
train_dataset = torch.load(DATASET_PATH + 'train_' + dataset_name + '.pt')
valid_dataset = torch.load(DATASET_PATH + 'valid_' + dataset_name + '.pt')

# Traininig and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Training

LEARNING_RATE = 0.0001
EPOCHS = 20
model_name = 'unet_unetr_lr_loss'

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("Number of parameters: ", sum(p.numel() for p in model.parameters() if p.requires_grad))

loss_manager = LossManager()

for epoch in range(EPOCHS):

    print(f'Epoch {epoch +1}/{EPOCHS}')
    for (train_img, train_mask), (valid_img, valid_mask) in tqdm(zip(train_dataloader, valid_dataloader)):
        # Calculate loss on validation set
        if validation_loss:
            with torch.no_grad():
                valid_img = valid_img.to(device=DEVICE)
                valid_mask = valid_mask.to(device=DEVICE)
                
                loss = criterion(model(valid_img), valid_mask)
                loss_manager.add(loss.item())

        train_img = train_img.to(device=DEVICE)
        train_mask = train_mask.to(device=DEVICE) # dim : (batch_size, 224, 224)
        # prediction
        mask_pred = model(train_img)  # dim : (batch_size, 10, 224, 224)

        # Calculate loss
        loss = criterion(mask_pred, train_mask)
            
            
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    #tracker.epoch_end()
    print(f'Loss: {loss.item()}')
    loss_manager.epoch_end()

torch.save(
    {   
        'input_size':INPUT_SIZE,
        'epochs':EPOCHS,
        'learning_rate':LEARNING_RATE,
        'batch_size':BATCH_SIZE,
        'num_classes':NUM_CLASSES,
        'model_state_dict':model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict(),
        'loss':loss,
        'datasets': dataset_name,
        'model_class': smp.Unet,
        'kwargs':kwargs
        }, MODEL_PATH + model_name + '.pt'
    )
file = open('results/loss/' + f'{model_name}_loss.txt', 'w')
for elt in loss_manager.losses:
    file.write(str(float(elt)) + '\n')
file.close()

Number of parameters:  23749546
Epoch 1/20


  0%|          | 0/438 [00:00<?, ?it/s]


NameError: name 'valid_img' is not defined